In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt
from string import Template
import os
import re
from collections import defaultdict
import random
import json
import asyncio
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline



In [2]:
from datetime import datetime, timedelta

dateString = "2013/12/21"
dateFormatter = "%Y/%m/%d"
datetime.strptime(dateString, dateFormatter)
test_symbols = ['2330', '3008', '2317', '2412']
DEBUG = False

loop = asyncio.get_event_loop()

In [3]:
# async pipline Test brokerId fetch  
async def fetch_brokerId():
    URL = 'https://fubon-ebrokerdj.fbs.com.tw/z/js/zbrokerjs.djjs'
    
    resp = requests.get(URL)
    resp.encoding = 'cp950' # 中文
    return resp.text
  
async def post_brokerId():
    global BkrMap
    text =  await fetch_brokerId()
    reBkr = re.compile(r".*g_BrokerList = \'(.*)\'", re.MULTILINE|re.DOTALL)
    reBkr2Id = re.compile(r"(.*),(.*)", re.DOTALL)
    BkrGrps = reBkr.match(text).group(1).split(';') # slice to broker groups
    BkrMap = defaultdict(lambda: defaultdict(str))

    for bkrgrp in BkrGrps:
        bkrlist = bkrgrp.split('!') # slice to sub broker
        for i, bkr in enumerate(bkrlist):
            bmt = reBkr2Id.match(bkr)
            if bmt:
                top_bkr = bmt.group(2) if i == 0 else top_bkr
                BkrMap[top_bkr].update({bmt.group(2): bmt.group(1)}) #group(1) = b

    # debug            
    #for k in sorted(BkrMap.keys())[:1]:
    #    print(BkrMap[k])
loop = asyncio.get_event_loop()
loop.run_until_complete(post_brokerId())
#post_brokerId()

In [4]:
# random pick BHID, SBHIDS
def random_pick_BHIDs():
    global BkrMap
    test_BHIDS = list(BkrMap[random.choice(list(BkrMap.keys()))].items())
    print (test_BHIDS)
    BHID = test_BHIDS[0][1]
    SBHID = random.choice(test_BHIDS[1:])[1] if len(test_BHIDS) > 2 else BHID # if have sub SBHIDS
    #print (BHID, SBHID)
    return (BHID, SBHID)
    

In [5]:
BHID = '5380' # 台新
SBHID = '0035003300380041' # 台新建北

if not DEBUG:
    BHID, SBHID = random_pick_BHIDs()
    
print (BHID, SBHID)

[('臺銀', '1040'), ('臺銀-民權', '1043'), ('臺銀-金山', '0031003000340044'), ('臺銀-高雄', '0031003000340043'), ('臺銀-新竹', '1045'), ('臺銀-臺中', '0031003000340041'), ('臺銀-臺南', '1042'), ('臺銀-鳳山', '1041')]
1040 0031003000340043


In [6]:
#http://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=2330&BHID=9800&b=9800
# get broker stock position detail put/call volume
def get_broker_sub_symbol_position(BHID, SBHID, symbol):
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=$SYMBOL&BHID=$BHID&b=$SBHID')
    URLPar = {
        'SYMBOL': symbol,
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 't01'})[0]
    values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
    ddates = list(map(lambda x: datetime.strptime(str(x.text), dateFormatter), table.find_all('td', {'class': 't4n0'})))
    #print (values)
    #print (len(values))
    calls = [values[i] for i in range(0, len(values), 4)]  #買
    puts = [values[i+1] for i in range(0, len(values), 4)] #賣
    BkrPutCall = {
        'date': ddates,
        'symbol': symbol,
        'puts': puts,
        'calls': calls
    }
    return  pd.DataFrame(BkrPutCall)

In [7]:
symbol = '2330'
if not DEBUG:
    symbol = random.choice(test_symbols)

BkrPutCallpf = get_broker_sub_symbol_position(BHID, SBHID, symbol)
BkrPutCallpf[:4]

,date,symbol,puts,calls
0,2021-03-05,3008,0,0
1,2021-03-04,3008,0,0
2,2021-03-03,3008,0,0
3,2021-03-02,3008,0,0


In [8]:
#https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=1440&b=1440&c=E&d=1
def get_broker_all_symbol_position(BHID, SBHID):
    
    # parser GenLink2stk('AS3481','群創');
    reSymbol2Id = re.compile(r".*GenLink2stk\(\'AS(.*)\',\'(.*)\'\);", re.M|re.DOTALL)  
    
    #parser <a href="javascript:Link2Stk('00677U');">00677U富邦VIX</a>
    # no support yet
    #reSymbol2Id2 = re.compile(r".*Link2Stk\(\'(.*)\'\);\"\>(.*)\<\/a\>", re.M|re.DOTALL)
    
    def get_symbol(text): 
        smt = reSymbol2Id.match(text)
        if smt:
            return smt.group(1)
            
    def get_puts_calls(table, Bkrlist):
        values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
        symbols = list(map(lambda x:  get_symbol(str(x.text)), table.find_all('td', {'class': 't4t1'})))
        
        for i, symbol in enumerate(symbols):
            if symbol == None:
                continue
            Bkrlist.append({
            'symbol': symbol,
            'calls': values[i*3+0], #買
            'puts': values[i*3+1]   #賣
            })
        
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=$BHID&b=$SBHID&c=E&d=1')
    URLPar = {
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    tables = soup.find_all('table', {'class': 't0'})
    Bkrlist = list()
    get_puts_calls(tables[0], Bkrlist) # 買超
    get_puts_calls(tables[1], Bkrlist) # 賣超
    
    return pd.DataFrame(Bkrlist)

In [9]:
BHID2Symbolpf = get_broker_all_symbol_position(BHID, SBHID)
BHID2Symbolpf.head()

,symbol,calls,puts
0,2014,248,61
1,2303,377,225
2,2409,245,95
3,6015,90,0
4,1305,74,8


In [34]:
import asyncio
import motor.motor_asyncio
from datetime import date, datetime
import functools
import pprint
import sys

sys.path.insert(0, os.path.abspath('../'))
import abupy
from abupy.UtilBu.ABuDateUtil import str_to_datetime

In [11]:
client = motor.motor_asyncio.AsyncIOMotorClient('mongodb://localhost:27017')
loop = asyncio.get_event_loop()

In [12]:
# spider cmd
#%scrapy crawl StockHisTraderSpider --logfile test.log
yy, mm, dd = date.today().isoformat().split("-")[0:3]
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
cdate = '2021-03-05'
dd = str_to_datetime(cdate)
symbol = '3008'

In [13]:
db = client['symbol-db']
collection = db['StockHisTrader']

async def do_find_StockHisTrader_one():
    document = await db['StockHisTrader'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    

loop.run_until_complete(do_find_StockHisTrader_one())

{'_id': ObjectId('60438c2bb2ddf58eaad3f307'),
 'date': datetime.datetime(2021, 3, 5, 0, 0),
 'symbol': '3008',
 'traders': [{'call': 64.0, 'put': 15.0, 'tdname': '新加坡商瑞銀', 'weight': 8.36},
             {'call': 0.0, 'put': 92.0, 'tdname': '台灣匯立證券', 'weight': 15.7},
             {'call': 37.0, 'put': 0.0, 'tdname': '華南永昌', 'weight': 6.31},
             {'call': 20.0, 'put': 94.0, 'tdname': '瑞士信貸', 'weight': 12.63},
             {'call': 33.0, 'put': 0.0, 'tdname': '凱基-三多', 'weight': 5.63},
             {'call': 0.0, 'put': 24.0, 'tdname': '元富-台中', 'weight': 4.1},
             {'call': 49.0, 'put': 21.0, 'tdname': '摩根大通', 'weight': 4.78},
             {'call': 5.0, 'put': 17.0, 'tdname': '美商高盛', 'weight': 2.05},
             {'call': 32.0, 'put': 5.0, 'tdname': '凱基-台北', 'weight': 4.61},
             {'call': 0.0, 'put': 12.0, 'tdname': '永豐金證券', 'weight': 2.05},
             {'call': 23.0, 'put': 0.0, 'tdname': '元富', 'weight': 3.92},
             {'call': 1.0, 'put': 11.0, 'tdname': '港商野村

In [14]:
async def do_find_StockHisData_one():
    document = await db['StockHisData'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    
loop.run_until_complete(do_find_StockHisData_one())

{'C': 3360.0,
 'H': 3415.0,
 'L': 3310.0,
 'O': 3310.0,
 'V': 604.0,
 '_id': ObjectId('6044c5ffc0a0fcadee5d8033'),
 'date': datetime.datetime(2021, 3, 5, 0, 0),
 'symbol': '3008'}


In [15]:
async def do_find_StockMargion_one():
    document = await db['StockHisMargin'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    
loop.run_until_complete(do_find_StockMargion_one())

{'MagPurchBalance': 1562.0,
 'MagPurchBuy': 16.0,
 'MagPurchCashRepay': 1.0,
 'MagPurchLimit': 33535.0,
 'MagPurchRatio': 4.66,
 'MagPurchSell': 32.0,
 'ShortDivMagRatio': 4.29,
 'ShortSaleBalance': 67.0,
 'ShortSaleBuy': 2.0,
 'ShortSaleCashRepay': 1.0,
 'ShortSaleSell': 3.0,
 '_id': ObjectId('60438ee08f4b0855aee02397'),
 'date': datetime.datetime(2021, 3, 5, 0, 0),
 'symbol': '3008'}


In [17]:
yy, mm, dd = date.today().isoformat().split("-")[0:3]
#dd = 19
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
today = str_to_datetime(cdate)

C_MIN, C_MAX = 50, 1000 #close Min:Max
V_MAX = 500 # volume >= V_max
broker_sampleD = 30 # broker sample [n]Days
candidates = []

In [18]:

async def do_find_jump_spot_symbols():
    # test find max count of call symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight

      # find jump spot 
      {"$lookup": {
           "from": "StockHisData",
           "pipeline": [
              {"$match":{ "$expr": {"$and": [
                        { "$lte": ["$C", C_MAX ] },
                        { "$gte": ["$C", C_MIN ] },
                        { "$eq": ["$C", "$O"]},
                        { "$ne": ["$O", "$H"]},
                        { "$gte": ["$V", V_MAX]},
                        { "$lte" : ["$date", datetime.now() ]},
                        { "$gte" : ["$date", datetime.now() - timedelta(days=1) ]}
                       ]}}},
              {"$project": {"_id": 0 } } # no _id to next pipeline
           ],
           "as": "p0_item"
      }},
      {"$unwind":"$p0_item"}, 
      {"$match":{"$expr":{"$eq":["$p0_item.symbol", "$symbol"]}}},
      {"$match":{"$expr":{"$and":[
          { "$lte": ["$date", datetime.now() ] },
          { "$gte": ["$date", datetime.now() - timedelta(days=broker_sampleD) ] }
      ]}}},
      {"$unwind": "$traders" },
      {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "tD_C": { "$first": "$p0_item.C"},
                    "tD_O": { "$first": "$p0_item.O"},
                    "tD_H": { "$first": "$p0_item.H"},
                    "tD_L": { "$first": "$p0_item.L"},
                    "tD_V": { "$first": "$p0_item.V"},
                    "tD":   { "$first": "$p0_item.date"},
                    "broker_sampleD": { "$first": broker_sampleD }
                 }},
      {"$sort": { "tD": -1, "sum_calls": -1, "sum_puts": 1, "tD_C": -1, "tD_V": -1 } },  
      #{"$limit": 1 }     
      ]
    collect = db['StockHisTrader'].aggregate(pipeline)
    async for cursor in collect: # clean cursor ptr and store to list
        #pprint.pprint(cursor)
        candidates.append(cursor)
        
loop.run_until_complete(do_find_jump_spot_symbols())

In [19]:
test_symbols = []
for it in candidates:
    if it['_id']['symbol'] not in test_symbols:
        test_symbols += [it['_id']['symbol']] 

test_symbols[:10]

[]

In [20]:
short_sampleD = 10
long_sampleD = 30

yy, mm, dd = date.today().isoformat().split("-")[0:3]
dd = 19
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
today = str_to_datetime(cdate)

symbol = '3008'
short_endD = today #datetime.now()
short_startD = datetime.now() - timedelta(days=short_sampleD)    
long_endD = datetime.now()
long_startD = datetime.now() - timedelta(days=long_sampleD)  

In [103]:
async def get_symbol_OHIV_df(symbol, startD, endD):
    pipeline = [
        {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
         {"$project" : { "_id": 0 } }
        ]
    collect = db['StockHisData'].aggregate(pipeline)
    items = await collect.to_list(100)
    df = pd.DataFrame(items)
    df = df.set_index('date')
    return df
 
df0 = loop.run_until_complete(get_symbol_OHIV_df(symbol, short_startD, short_endD))
df0

,O,H,L,C,V,symbol
date,,,,,,
2021-02-26,3320.0,3365.0,3280.0,3295.0,1459.0,3008
2021-03-02,3380.0,3400.0,3340.0,3345.0,849.0,3008
2021-03-03,3345.0,3440.0,3315.0,3440.0,878.0,3008
2021-03-04,3420.0,3430.0,3355.0,3375.0,548.0,3008
2021-03-05,3310.0,3415.0,3310.0,3360.0,604.0,3008


In [104]:
async def get_symbol_Margin_df(symbol, startD, endD):
    pipeline = [
        {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
         {"$project" : { "_id": 0 } }
        ]
    collect = db['StockHisMargin'].aggregate(pipeline)
    items = await collect.to_list(100)
    df = pd.DataFrame(items)
    df = df.set_index('date')
    return df

df1 = loop.run_until_complete(get_symbol_Margin_df(symbol, short_startD, short_endD))
df1


,MagPurchBuy,MagPurchSell,MagPurchCashRepay,MagPurchBalance,MagPurchLimit,MagPurchRatio,ShortSaleSell,ShortSaleBuy,ShortSaleCashRepay,ShortSaleBalance,ShortDivMagRatio,symbol
date,,,,,,,,,,,,
2021-03-03,14.0,52.0,0.0,1594.0,33535.0,4.75,2.0,1.0,0.0,67.0,4.20,3008
2021-03-05,16.0,32.0,1.0,1562.0,33535.0,4.66,3.0,2.0,1.0,67.0,4.29,3008


In [91]:
async def do_find_symbol_top_calls(symbol, startD, endD, rank=5):
    # test find max count of call symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight
       {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
        {"$unwind": "$traders" },
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" } }  },
        {"$sort": { "sum_calls": -1, "sum_puts": 1 } },
        {"$limit": rank }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline)
    tops = []
    async for cursor in collect: # clean cursor ptr and store to list
        tops.append(cursor['_id']['tdname'])
    return tops


tops = loop.run_until_complete(do_find_symbol_top_calls(symbol, short_startD, short_endD))   
tops

['台灣摩根士丹利', '摩根大通', '美林', '凱基-台北', '花旗環球']

In [105]:
async def get_symbol_top_calls_df(symbol, startD, endD, tradernm, rank=0):
    # test find max count of call symbols at sample time window
    put  = "Top{0}_put".format(rank)
    call = "Top{0}_call".format(rank)
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight
       {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
        {"$unwind": "$traders" },
        {"$match":{ "$expr": {"$and": [
                { "$eq": ["$traders.tdname", tradernm ] }
            ]}}},
        {"$project" : { "_id": 0, "date": "$date", put: "$traders.put", call: "$traders.call", "symbol": "$symbol" } }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline)
    items = await collect.to_list(100)
    df = pd.DataFrame(items)
    df = df.set_index('date')
    return df
        
df2 = loop.run_until_complete(get_symbol_top_calls_df(symbol, short_startD, short_endD, tops[0], 0))   
df2

,Top0_put,Top0_call,symbol
date,,,
2021-02-26,19.0,30.0,3008
2021-03-02,35.0,195.0,3008
2021-03-03,80.0,102.0,3008
2021-03-04,21.0,40.0,3008


In [123]:

df1 = df1[['MagPurchRatio','ShortDivMagRatio']]
df1 = df1.rename(columns={"MagPurchRatio": "MagCallR", "ShortDivMagRatio": "ShortDivMagR"})
df2 = df2[['Top0_put', 'Top0_call']]
         
df_final = pd.concat(
    [df0, df1, df2],
    axis=1,
    join='outer' #inner
).fillna(0)
df_final


,O,H,L,C,V,symbol,MagCallR,ShortDivMagR,Top0_put,Top0_call
date,,,,,,,,,,
2021-02-26,3320.0,3365.0,3280.0,3295.0,1459.0,3008,0.00,0.00,19.0,30.0
2021-03-02,3380.0,3400.0,3340.0,3345.0,849.0,3008,0.00,0.00,35.0,195.0
2021-03-03,3345.0,3440.0,3315.0,3440.0,878.0,3008,4.75,4.20,80.0,102.0
2021-03-04,3420.0,3430.0,3355.0,3375.0,548.0,3008,0.00,0.00,21.0,40.0
2021-03-05,3310.0,3415.0,3310.0,3360.0,604.0,3008,4.66,4.29,0.0,0.0


In [108]:
df_final

O       H       L       C       V symbol
     date                                                     
date 2021-02-26  3320.0  3365.0  3280.0  3295.0  1459.0   3008
     2021-03-02  3380.0  3400.0  3340.0  3345.0   849.0   3008
     2021-03-03  3345.0  3440.0  3315.0  3440.0   878.0   3008
     2021-03-04  3420.0  3430.0  3355.0  3375.0   548.0   3008
     2021-03-05  3310.0  3415.0  3310.0  3360.0   604.0   3008

In [100]:
test_symbols = []
for candidate in candidates:
    symbol = candidate['_id']['symbol'] 
    tD = candidate['tD']
    if symbol in test_symbols:
        continue
    test_symbols += [symbol]
    short_df = loop.run_until_complete(get_symbol_df(symbol, short_startD, short_endD))
    short_df['C'] = short_df['C'].astype(float)
    #print (short_df.index[-1])
    n_short_df = short_df.sort_values(by='C', ascending=False)
    if short_df.index[-1] in n_short_df.index[-3:-1]:
        print (symbol, short_df.index[-3:-1])
